In [ ]:
import json
import os

import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
import dash_reusable_components as drc
import dash_bootstrap_components as dbc
import flask
import pandas as pd
import io
from io import StringIO
import base64
import datetime
import dash_table
from textwrap import dedent as d
import plotly.graph_objs as go
import networkx as nx
# Load extra layouts
cyto.load_extra_layouts()


asset_path = os.path.join(
    os.path.dirname(os.path.abspath('__file__')),
    '..', 'assets'
)
server = flask.Flask(__name__)
app = dash.Dash(server=server, assets_folder=asset_path)
#server = app.server
#app = dash.Dash(__name__)
#server = app.server

# ###################### DATA PREPROCESSING ######################
# Load data
network_data = nx.read_gpickle('graph.gpickle')

# We select the first 750 edges and associated nodes for an easier visualization
edges = list(network_data.edges())
nodes = set()
global flag
trace1 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[20, 14, 23],
    name='SF Zoo'
)
trace2 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[12, 18, 29],
    name='LA Zoo'
)
trace3 = go.Bar(
    x=['giraffes', 'orangutans', 'monkeys'],
    y=[12, 18, 29],
    name='LA Zoo'
)

following_node_di = {}  # user id -> list of users they are following
following_edges_di = {}  # user id -> list of cy edges starting from user id

followers_node_di = {}  # user id -> list of followers (cy_node format)
followers_edges_di = {}  # user id -> list of cy edges ending at user id

cy_edges = []
cy_nodes = []
#dos=nx.get_node_attributes(G,'dosage')
#lin=nx.get_node_attributes(G,'line')
for edge in edges:
    source = edge[0]
    target = edge[1]

    cy_edge = {'data': {'id': source+target, 'source': source, 'target': target}}
    cy_target = {"data": {"id": target, "label":str('NAME:'+target.split("_")[1]+' , LINE:'+str(lin[target])+', DOSAGE:'+str(dos[target]))}}
    cy_source = {"data": {"id": source, "label": str('NAME:'+source.split("_")[1]+' , LINE:'+str(lin[source])+' , DOSAGE:'+str(dos[source]))}}

    if source not in nodes:
        nodes.add(source)
        cy_nodes.append(cy_source)
    if target not in nodes:
        nodes.add(target)
        cy_nodes.append(cy_target)

    # Process dictionary of following
    if not following_node_di.get(source):
        following_node_di[source] = []
    if not following_edges_di.get(source):
        following_edges_di[source] = []

    following_node_di[source].append(cy_target)
    following_edges_di[source].append(cy_edge)

    # Process dictionary of followers
    if not followers_node_di.get(target):
        followers_node_di[target] = []
    if not followers_edges_di.get(target):
        followers_edges_di[target] = []

    followers_node_di[target].append(cy_source)
    followers_edges_di[target].append(cy_edge)

genesis_node = cy_nodes
#genesis_node['classes'] = "genesis"
default_elements = cy_nodes
regimen=[]
start=[]
end=[]
default_stylesheet = [
    {
        "selector": 'node',
        'style': {
            "opacity": 0.65,
            'z-index': 9999
        }
    },
    {
        "selector": 'edge',
        'style': {
            "curve-style": "bezier",
            "opacity": 0.45,
            'z-index': 5000
        }
    },
    {
        'selector': '.followerNode',
        'style': {
            'background-color': '#0074D9',
            
        }
    },
    {
        'selector': '.followerEdge',
        "style": {
            "mid-target-arrow-color": "blue",
            "mid-target-arrow-shape": "vee",
            "line-color": "#0074D9"
        }
    },
    {
        'selector': '.followingNode',
        'style': {
            'background-color': '#FF4136',
            
        }
    },
    {
        'selector': '.followingEdge',
        "style": {
            "mid-target-arrow-color": "red",
            "mid-target-arrow-shape": "vee",
            "line-color": "#FF4136",
        }
    },
    {
        "selector": '.genesis',
        "style": {
            'background-color': '#B10DC9',
            "border-width": 2,
            "border-color": "purple",
            "border-opacity": 1,
            "opacity": 1,

            "label": "data(label)",
            "color": "#B10DC9",
            "text-opacity": 1,
            "font-size": 12,
            'z-index': 9999
        }
    },
    {
        'selector': ':selected',
        "style": {
            "border-width": 2,
            "border-color": "black",
            "border-opacity": 1,
            "opacity": 1,
            "label": "data(label)",
            "color": "black",
            "font-size": 12,
            'z-index': 9999
        }
    }
]

# ################################# APP LAYOUT ################################
styles = {
    'json-output': {
        'overflow-y': 'scroll',
        'height': 'calc(50% - 25px)',
        'border': 'thin lightgrey solid'
    },
    'tab': {'height': 'calc(100% - 100px)'},
    'tabs1': {'height': 'calc(100% - 100px)'}
}

app.layout = html.Div([
    
    html.Div(className='eight columns', children=[dcc.Tabs(id='tabs1',vertical = False, children=[
            dcc.Tab(label='Upload Data',children=[html.Div([
    dcc.Upload(
        id='upload-data',
        children=html.Div([
            'Market share ',
            html.A(' data')
        ]),
        style={
            'width': '40%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px',
            'box-shadow': '5px 10px #888888'
        },
        # Allow multiple files to be uploaded
        multiple=True
    )
    
]),html.Div([
    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Optimization  ',
            html.A(' Output')
        ]),
        style={
            'width': '25%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '120px',
            'box-shadow': '5px 10px #888888'
        },
        # Allow multiple files to be uploaded
        multiple=True
    )
]),html.Div([
    dcc.Upload(
        id='upload-data3',
        children=html.Div([
            'hemonc ',
            html.A(' data')
        ]),
        style={
            'width': '17%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '180px',
            'box-shadow': '5px 10px #888888'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload3'),
]),html.Div([
    dcc.Upload(
        id='upload-data4',
        children=html.Div([
            'Hemonc graph',
            html.A(' data')
        ]),
        style={
            'width': '10%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'solid',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '240px',
            'box-shadow': '5px 10px #888888'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-data-upload4'),
])
                                                 ]),
            dcc.Tab(label='Output',children=[html.Div([
    dcc.Graph(
        id='basic-interactions',
        figure={
            'data': [
                {
                    'x': regimen,
                    'y': start,
                    'text': ['a', 'b', 'c', 'd'],
                    'customdata': ['c.a', 'c.b', 'c.c', 'c.d'],
                    'name': 'Trace 1',
                    'mode': 'markers',
                    'marker': {'size': 12}
                },
                {
                    'x': regimen,
                    'y': end,
                    'text': ['w', 'x', 'y', 'z'],
                    'customdata': ['c.w', 'c.x', 'c.y', 'c.z'],
                    'name': 'Trace 2',
                    'mode': 'markers',
                    'marker': {'size': 12}
                }
            ],
            'layout': {
                'clickmode': 'event+select'
            }
        }
    ),

    html.Div(className='row', children=[
        html.Div([
            dcc.Markdown(d("""
                **Hover Data**

                Mouse over values in the graph.
            """)),
            html.Pre(id='hover-data')
        ], className='three columns'),

        html.Div([
            dcc.Markdown(d("""
                **Click Data**

                Click on points in the graph.
            """)),
            html.Pre(id='click-data'),
        ], className='three columns'),

        html.Div([
            dcc.Markdown(d("""
                **Selection Data**

                Choose the lasso or rectangle tool in the graph's menu
                bar and then select points in the graph.

                Note that if `layout.clickmode = 'event+select'`, selection data also 
                accumulates (or un-accumulates) selected data if you hold down the shift
                button while clicking.
            """)),
            html.Pre(id='selected-data'),
        ], className='three columns'),

        html.Div([
            dcc.Markdown(d("""
                **Zoom and Relayout Data**

                Click and drag on the graph to zoom or click on the zoom
                buttons in the graph's menu bar.
                Clicking on legend items will also fire
                this event.
            """)),
            html.Pre(id='relayout-data'),
        ], className='three columns')
    ])
]),
                                            html.Div([
    dcc.Graph(id='bar_plot',
              figure=go.Figure(data=[trace1, trace2],
                               layout=go.Layout(barmode='stack'))
              )
    ]),html.Div([
    dcc.Graph(id='bar_plot2',
              figure=go.Figure(data=[trace3],
                               layout=go.Layout(barmode='stack'))
              )
    ])]),
            dcc.Tab(label='Model Output',children=[
    html.Div(id='output-image-upload3')]),
           dcc.Tab(label='Hemonc',children=[html.Div(className='eight columns', children=[
        cyto.Cytoscape(
            id='cytoscape',
            elements=default_elements,
            
            stylesheet=default_stylesheet,
            style={
                'height': '95vh',
                'width': '100%'
            }
        ),
        
        
        drc.NamedDropdown(
                    name='regimen',
                    id='dropdown-layout-1',
                    options=drc.DropdownOptionsList(*list(G.nodes())),
                    value='L0_PLATINUM,TAXANE',
                    clearable=False
                ),
               
        drc.NamedDropdown(
                    name='line',
                    id='dropdown-layout-2',
                    options=drc.DropdownOptionsList('random',
                        'L0',
                        'L1',
                        'L2',
                        'L3',
                        'L4',
                        'L5',
                        'L6',
                        'L7',
                        'LN',
                        'LM',
                        'L9'),
                    value='L0',
                    clearable=False
                ),
               
        drc.NamedDropdown(
                    name='depth',
                    id='dropdown-layout-3',
                    options=drc.DropdownOptionsList('random',
                        '1',
                        '2',
                        '3',
                        '4',
                        '5',
                        '6',
                        '7',
                        '8',
                        '9',
                        '10',
                        '11'),
                    value='1',
                    clearable=False
                ),
        html.P(id='cytoscape-tapNodeData-output'),
        html.P(id='cytoscape-tapEdgeData-output'),
        html.P(id='cytoscape-mouseoverNodeData-output'),
        html.P(id='cytoscape-mouseoverEdgeData-output')
        
    ]),

    html.Div(className='four columns', children=[
        dcc.Tabs(id='tabs', children=[
            dcc.Tab(label='Control Panel', children=[
                drc.NamedDropdown(
                    name='Layout',
                    id='dropdown-layout',
                    options=drc.DropdownOptionsList(
                        'random',
                        'grid',
                        'circle',
                        'concentric',
                        'breadthfirst',
                        'cose',
                        'cose-bilkent',
                        'dagre',
                        'cola',
                        'klay',
                        'spread',
                        'euler'
                    ),
                    value='cola',
                    clearable=False
                )
            ])

            
        ]),

    ])]),
                   ]
                 
                )]),
    
    
])


# ############################## CALLBACKS ####################################
def parse_contents_1(contents, filename):
    
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'gpickle' in filename:
            # Assume that the user uploaded an excel file
            df = nx.read_gpickle(io.StringIO(decoded.decode()))   
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return df
def parse_image(contents, filename, date):
    return html.Div([
        html.H5(filename.split('.')[0]),
        #html.H6(datetime.datetime.fromtimestamp(date)),

        # HTML images accept base64 encoded strings in the same format
        # that is supplied by the upload
        html.Img(src=contents),
        html.Hr(),
        html.Div('')
#         html.Pre(contents[0:200] + '...', style={
#             'whiteSpace': 'pre-wrap',
#             'wordBreak': 'break-all'
#         }
#                 )
    ])
def parse_contents(contents, filename,date):
    content_type, content_string = contents.split(',')

    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            # Assume that the user uploaded a CSV file
            df = pd.read_csv(
                io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename:
            # Assume that the user uploaded an excel file
            df = pd.read_excel(io.BytesIO(decoded))
        elif 'gpickle' in filename:
            # Assume that the user uploaded an excel file
            df = nx.read_gpickle(io.StringIO(decoded.decode()))   
    except Exception as e:
        print(e)
        return html.Div([
            'There was an error processing this file.'
        ])

    return html.Div([
        html.H5(filename),
        html.H6(datetime.datetime.fromtimestamp(date)),

        dash_table.DataTable(
            data=df.to_dict('records'),
            columns=[{'name': i, 'id': i} for i in df.columns]
        ),

        html.Hr(),  # horizontal line

        # For debugging, display the raw contents provided by the web browser
        html.Div('Raw Content'),
        html.Pre(contents[0:200] + '...', style={
            'whiteSpace': 'pre-wrap',
            'wordBreak': 'break-all'
        })
    ])
@app.callback(Output('output-image-upload3', 'children'),
               [Input('upload-image', 'contents')],
               [State('upload-image', 'filename'),
               State('upload-image', 'last_modified')])
def update_output_1(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
             parse_image(c, n,d) for c, n,d in
             zip(list_of_contents, list_of_names,list_of_dates)]
        return children
@app.callback(Output('basic-interactions', 'figure'),
              [Input('upload-data', 'contents')],
              [State('upload-data', 'filename'),
               State('upload-data', 'last_modified')])
def update_output_2(list_of_contents, list_of_names,list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents_1(c, n) for c, n in
            zip(list_of_contents, list_of_names)]
        
        regimen=children[0]["therapy_standardized"]
        start=children[0]["Regimen Start Date"]
        end=children[0]["Regimen End Date"]
        figure1={
            'data': [
                {
                    'x': regimen,
                    'y': start,
                    'text': ['a', 'b', 'c', 'd'],
                    'customdata': ['c.a', 'c.b', 'c.c', 'c.d'],
                    'name': 'Trace 1',
                    'mode': 'markers',
                    'marker': {'size': 12}
                },
                {
                    'x': regimen,
                    'y': end,
                    'text': ['w', 'x', 'y', 'z'],
                    'customdata': ['c.w', 'c.x', 'c.y', 'c.z'],
                    'name': 'Trace 2',
                    'mode': 'markers',
                    'marker': {'size': 12}
                }
            ],
            'layout': {
                'clickmode': 'event+select'
            }
        }
        return figure1

@app.callback(Output('output-data-upload3', 'children'),
              [Input('upload-data3', 'contents')],
              [State('upload-data3', 'filename'),
               State('upload-data3', 'last_modified')])
def update_output_3(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n,d) for c, n,d in
            zip(list_of_contents, list_of_names,list_of_dates)]
        return children
@app.callback(Output('output-data-upload4', 'children'),
              [Input('upload-data4', 'contents')],
              [State('upload-data4', 'filename'),
               State('upload-data4', 'last_modified')])
def update_output_4(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n,d) for c, n,d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children
@app.callback(Output('cytoscape-mouseoverNodeData-output', 'children'),
                  [Input('cytoscape', 'mouseoverNodeData')])
def displayTapNodeData(data):
    if data:
        return "You recently hovered over: " + data['label']


@app.callback(Output('cytoscape', 'layout'),
              [Input('dropdown-layout', 'value')])
def update_cytoscape_layout(layout):
    return {'name': layout}


@app.callback(Output('cytoscape', 'elements'),
              [Input('dropdown-layout-1', 'value'),Input('dropdown-layout-2', 'value'),Input('dropdown-layout-3', 'value'),],
              [State('cytoscape', 'elements')])
def generate_elements(reg,line, depth,elements):
    
    for element in elements:
        if reg == element.get('data').get('id'):
            elements=[element]
            break
    line_=["L0","L1","L2","LN","LM","L3","L4","L5","L6","L7"]
    for i in range(int(depth)):
        followers_nodes = followers_node_di.get(elements[i].get('data').get('id'))
        followers_edges = followers_edges_di.get(elements[i].get('data').get('id'))

        if followers_nodes:
            for node in followers_nodes:
                node['classes'] = 'followerNode'
            elements.extend(followers_nodes)

        if followers_edges:
            for follower_edge in followers_edges:
                follower_edge['classes'] = 'followerEdge'
            elements.extend(followers_edges)



        following_nodes = following_node_di.get(elements[i].get('data').get('id'))
        following_edges = following_edges_di.get(elements[i].get('data').get('id'))

        if following_nodes:
            for node in following_nodes:
                node['classes'] = 'followingNode'
                elements.append(node)

        if following_edges:
            for follower_edge in following_edges:
                follower_edge['classes'] = 'followingEdge'
            elements.extend(following_edges)
        

    return elements

#@app.callback(Output('cytoscape', 'elements'),
              #[Input('dropdown-layout-1', 'value')])
#def update_layout_1(layout):
    #target = {"data": {"id": layout, "label": layout}}
    #return {'elements': target}

if __name__ == '__main__':
    app.run_server()